In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import matplotlib.pyplot as plt
import random

In [2]:
class TextureDataset(Dataset):
    def __init__(self, diff_dir, disp_dir, nor_dir, rough_dir,transform=None):
        self.diff = diff_dir
        self.disp = disp_dir
        self.nor = nor_dir
        self.rough= rough_dir
        self.transform = transform
    def __len__(self):
        return len(os.listdir(self.diff))
    def __getitem__(self,idx):
        diff_img = Image.open(self.diff+"\\"+os.listdir(self.diff)[idx])
        disp_img = Image.open(self.disp+"\\"+os.listdir(self.disp)[idx])
        nor_img = Image.open(self.nor+"\\"+os.listdir(self.nor)[idx])
        rough_img = Image.open(self.rough+"\\"+os.listdir(self.rough)[idx])
        img_list = [diff_img, disp_img, nor_img, rough_img]
        seed = np.random.randint(2147483647)
        
        if self.transform is not None:
            for i, img in enumerate(img_list):
                random.seed(seed)
                torch.manual_seed(seed)
                img_list[i] = self.transform(img)

        return img_list[0], img_list[1], img_list[2], img_list[3]

In [3]:
data_transform = transforms.Compose([
        transforms.RandomCrop(900),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Resize(256),
        transforms.ToTensor(),
    ])

In [4]:
diff_dir = ".\\Data\\diff"
disp_dir = ".\\Data\\disp"
nor_dir = ".\\Data\\nor"
rough_dir = ".\\Data\\rough"
dataset = TextureDataset(diff_dir, disp_dir, nor_dir, rough_dir, transform = data_transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle =True)
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle = True)

In [5]:
from resnet import ResNet

device = torch.device("cpu")

model = ResNet().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

criterion = nn.MSELoss().to(device)

In [6]:
diff, disp, nor, rough = next(iter(train_dataloader))
out = model(diff)

In [7]:
out.shape

torch.Size([32, 1, 256, 256])

In [8]:
epochs = 20

for epoch in range(epochs):
    loss = 0
    loss2 = 0
    for x,y,_,_ in train_dataloader:
        x= x.to(device)
        y= y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        
        train_loss = criterion(outputs, y)
        train_loss.backward()
        optimizer.step()
        loss += train_loss.item()
    for x,y in test_dataloader:
        x= x.to(device)
        y= y.to(device)
        outputs = model(x)
        test_loss = criterion(outputs, y)
        loss2 += test_loss.item() 
        
    loss = loss / len(train_dataloader)
    loss2 = loss2 / len(test_dataloader)
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}, testloss = {:.6f}".format(epoch + 1, epochs, loss,loss2))

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1073741824 bytes. Buy new RAM!


In [7]:
x = torch.tensor([1,2,3,4])

In [8]:
x = x.to(device)